In [1]:
import csv
from collections import defaultdict
from dotenv import load_dotenv
import os
import requests


In [2]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [3]:
import pickle

def pickling(path, data):
    file = open(path,'wb')
    pickle.dump(data,file)

def unpickling(path):
    file = open(path, 'rb')
    b = pickle.load(file)
    return b

In [4]:
def get_movie_id(movie_name, movie_year):
    """Gets the movie_id from TMDB using the movie_name and year

    Args:
        movie_name (str): Movie's name
        movie_year (str): Movie's release year

    Returns:
        int: Movie TMDB id
    """

    r = requests.get(f'https://api.themoviedb.org/3/search/movie?api_key={API_KEY}&language=en-US&page=1&include_adult=false', params={'query':movie_name, 'year':movie_year})
    movie_id = r.json()['results'][0]['id']
    return movie_id

def get_movie_actors(movie_name, movie_year, limit=20):
    """Returns the top 'limit' movie actors

    Args:
        movie_name (str): Movie's name
        movie_year (str): Movie's release year
        limit (int, optional): Actors limit. Defaults to 20.

    Returns:
        str[]: A list of movie actors
    """

    actors = []

    movie_id = get_movie_id(movie_name, movie_year)
    r = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={API_KEY}&language=en-US&')

    actors_json = r.json()['cast'][:limit]
    for actor in actors_json:
        actors.append(actor['name'])

    return actors

In [5]:
actors= get_movie_actors("mother !", "2017")
actors

['Jennifer Lawrence',
 'Javier Bardem',
 'Ed Harris',
 'Michelle Pfeiffer',
 'Brian Gleeson',
 'Domhnall Gleeson',
 'Jovan Adepo',
 'Amanda Chiu',
 'Patricia Summersett',
 'Eric Davis',
 'Raphael Grosz-Harvey',
 'Emily Hampshire',
 'Abraham Aronofsky',
 'Luis Oliva',
 'Stephanie Ng Wan',
 'Chris Gartin',
 'Stephen McHattie',
 'Ambrosio De Luca',
 'Gregg Bello',
 'Arthur Holden']

In [6]:
%%time
# get movies name and year from csv
movies_list = []
with open('top_movies_list.csv', newline='') as file:
    reader = csv.reader(file)
    for idx, movie in enumerate(reader):
        movies_list.append(movie)
        
print(len(movies_list))

1200
CPU times: total: 31.2 ms
Wall time: 3.99 ms


In [7]:
movies_list[0][1]

'Avatar'

In [11]:
actors_data = unpickling("data.pckl")

In [13]:
movies_list[0]

['1', 'Avatar', '2009', '$2881837181']

In [15]:
%%time

actors_map = defaultdict(int)
actors_movies_map = defaultdict(list)

for idx, movie in enumerate(movies_list):
    try:
        # Get the movies actors
        movie_actors = actors_data[idx]

        # Loop through the actors and add the movies sale to them
        for actor in movie_actors:
            try:
                actors_map[actor] += int(movie[3][1:]) # 1: to remove the $ sign
            except:
                actors_map[actor] += 0
            actors_movies_map[actor].append(movie[1])
        
        # if idx >= 200:
        #     break
    except:
        print(idx, movie)

pickling("actors_map.pckl", actors_map)
pickling("actors_movies_map.pckl", actors_movies_map)

CPU times: total: 46.9 ms
Wall time: 57.8 ms


In [16]:
def print_actors_map(actors_map, actors_movies_map, limit=20):
    sorted_actors_map = dict(sorted(actors_map.items(), key=lambda x: x[1], reverse=True))
    idx = 0
    for actor_name, profit in sorted_actors_map.items():
        movies_list = actors_movies_map[actor_name]
        print(idx, actor_name, profit, movies_list)
        idx += 1
        if idx > limit:
            break

In [20]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np

def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n","<br>")))

def print_actors_map_pd(actors_map, actors_movies_map, limit=200):

    sorted_actors_map = dict(sorted(actors_map.items(), key=lambda x: x[1], reverse=True))

    data = []
    idx = 0
    for actor_name, profit in sorted_actors_map.items():
        movies_list = actors_movies_map[actor_name]
        data.append([actor_name, f"${profit:,}", "\n".join(movies_list)])
        idx += 1
        if idx > limit:
            break

    df = pd.DataFrame(data, columns=["Actor Name", "Profit", "Movies"])
    # df = df.style.set_properties(**{'text-align': 'left'})
    pretty_print(df)

In [21]:
print_actors_map_pd(actors_map, actors_movies_map)

,Actor Name,Profit,Movies
0,Samuel L. Jackson,"$16,338,817,882",The AvengersAvengers: Age of UltronIncredibles 2Spider-Man: Far from HomeCaptain MarvelJurassic ParkStar Wars: Episode III - Revenge of the SithCaptain America: The Winter SoldierStar Wars: Episode II - Attack of the ClonesThe IncrediblesIron Man 2Kong: Skull IslandDjango UnchainedKingsman: The Secret ServiceCaptain America: The First AvengerDie Hard with a VengeanceThe Legend of TarzanxXx: Return of Xander CageMiss Peregrine's Home for Peculiar ChildrenComing to AmericaTurboxXxUnbreakableGlassJumperPulp FictionS.W.A.T.Patriot Games
1,Robert Downey Jr.,"$13,952,302,902",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronIron Man 3Captain America: Civil WarSpider-Man: HomecomingIron Man 2Iron ManSherlock Holmes: A Game of ShadowsSherlock HolmesDolittleDue DateTropic Thunder
2,Benedict Cumberbatch,"$13,508,189,548",Avengers: EndgameAvengers: Infinity WarSpider-Man: No Way HomeThe Hobbit: The Battle of the Five ArmiesThe Hobbit: The Desolation of SmaugDoctor Strange in the Multiverse of MadnessThor: RagnarokDoctor StrangeThe GrinchStar Trek Into Darkness1917Penguins of MadagascarThe Imitation Game12 Years a SlaveWar Horse
3,Scarlett Johansson,"$13,473,169,463",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronCaptain America: Civil WarThe Jungle BookCaptain America: The Winter SoldierSingIron Man 2LucySing 2Black WidowThe Horse WhispererHe's Just Not That Into You
4,Alan Tudyk,"$12,357,426,928",Frozen IIFrozen IIFrozenTransformers: Dark of the MoonRogue One: A Star Wars StoryZootopiaIce Age: Continental DriftBig Hero 6MoanaRalph Breaks the InternetWreck-It RalphIce AgeAlvin and the Chipmunks: ChipwreckedMaze Runner: The Scorch TrialsLincolnEncantoKnocked Up
5,Idris Elba,"$11,767,250,635",Avengers: Infinity WarAvengers: Age of UltronFinding DoryZootopiaThe Jungle BookThor: RagnarokThor: Love and ThunderFast & Furious Presents: Hobbs & ShawThor: The Dark WorldThorPacific RimPrometheusSonic the Hedgehog 2Star Trek BeyondAmerican Gangster
6,Chris Hemsworth,"$11,741,117,528",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronThor: RagnarokThor: Love and ThunderThor: The Dark WorldThorSnow White and the HuntsmanStar TrekMen in Black: InternationalGhostbusters
7,Zoe Saldaña,"$11,434,686,068",AvatarAvengers: EndgameAvengers: Infinity WarGuardians of the Galaxy Vol. 2Guardians of the GalaxyPirates of the Caribbean: The Curse of the Black PearlStar Trek Into DarknessStar TrekStar Trek BeyondThe Terminal
8,Andy Serkis,"$11,305,428,948",Star Wars: Episode VII - The Force AwakensBlack PantherStar Wars: Episode VIII - The Last JediThe Lord of the Rings: The Return of the KingThe Lord of the Rings: The Two TowersThe Lord of the Rings: The Fellowship of the RingThe BatmanDawn of the Planet of the ApesKing KongWar for the Planet of the ApesRise of the Planet of the ApesThe Adventures of TintinFlushed Away
9,Chris Evans,"$11,179,294,976",Avengers: EndgameAvengers: Infinity WarThe AvengersAvengers: Age of UltronCaptain America: Civil WarCaptain America: The Winter SoldierCaptain America: The First AvengerFantastic FourKnives OutFantastic Four: Rise of the Silver SurferLightyear


## Main

In [ ]:

print_actors_map_pd(actors_map, actors_movies_map)